In [1]:
import os
import uuid
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# metrics calculation to calculate Precision and Recall
from matplotlib.pyplot import figure
from tensorflow.keras.metrics import Precision, Recall
import cv2 as cv
"""
 Using Functional API allows us to be specific in our creation of a NN
 as it can accept multiple input and out note stack(linear) like sequential model
"""
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, Layer,Dropout 
from tensorflow.keras.losses import BinaryCrossentropy

"""
# Model : to compile all the layers together
# Layer : is a class which allows us to create a specific layer in our Network
# Dense : fully connected layer
# MaxPooling : allows us to shrink the information which we got A technic to
reduce information in an image while minting features
Input : another base class to input the image 
# Flatten : take the output from the Previous layer an make it in lone dimension
"""


'\n# Model : to compile all the layers together\n# Layer : is a class which allows us to create a specific layer in our Network\n# Dense : fully connected layer\n# MaxPooling : allows us to shrink the information which we got A technic to\nreduce information in an image while minting features\nInput : another base class to input the image \n# Flatten : take the output from the Previous layer an make it in lone dimension\n'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')  # getting the different gpus on our machine
for gpu in gpus:  # looping throw them
    tf.config.experimental.set_memory_growth(gpu, True)  # to set our gpus to true

In [3]:
for gpu in gpus:
    print(gpu)

In [4]:
# Setup our paths
 
POS_PATH = os.path.join("data", "positive")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor")

In [5]:
#Make the directories

os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

In [6]:
"""wing repository data/negative
"""
for directory in os.listdir("lfw"):
    
    for file in os.listdir(os.path.join("lfw", directory)):
        EX_path = os.path.join("lfw", directory, file)
        NEW_path = os.path.join(NEG_PATH, file)
        os.replace(EX_path, NEW_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'lfw'

In [7]:
"""
Establish a connection to the Camera
"""


cap = cv.VideoCapture(0)
while cap.isOpened():
    # read(): return two values one as a boolean and a frame from the camera
    ret, frame = cap.read()

    # cut down frame to 250x250px
    frame = frame[120:120 + 250, 200:200 + 250, :]

    # Collect anchors
    if cv.waitKey(1) & 0XFF == ord("a"):
        imgname = os.path.join(ANC_PATH, f"{uuid.uuid1()}.jpg")
        cv.imwrite(imgname, frame)

    # Collecting positive
    if cv.waitKey(1) & 0XFF == ord("p"):
        imgname = os.path.join(POS_PATH, f"{uuid.uuid1()}.jpg")
        cv.imwrite(imgname, frame)

    # Now we will show the the video frame by frame
    cv.imshow("Image Collection", frame)

    # waitKey(1) will display a frame for 1 ms, after which display will be automatically closed.
    if cv.waitKey(1) & 0XFF == ord('q'):
        break

# Release the camera
cap.release()

# Close the image show frame
cv.destroyAllWindows()

In [8]:
# Data Augmentation 

def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img,min_jpeg_quality=90,max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100), np.random.randint(100)))
        data.append(img)
    return data                                                

In [9]:
# Testing 1
img_path = os.path.join(ANC_PATH, "4c3bd0a4-c001-11ed-a93f-ec2e980717f3.jpg")
img = cv.imread(img_path)
augmented_imgs = data_aug(img)

for image in augmented_imgs:
    cv.imwrite(os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1())), image.numpy())

In [ ]:
# Testing 2 
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv.imread(img_path)
    augmented_imgs = data_aug(img)
    
    for image in augmented_imgs:
        cv.imwrite(os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1())), image.numpy())

In [10]:
"""
Get Images Directories
tf.data.DataSet.list_files() : used to get a group of images in specific directory
create a pipeline for all the files in specific directory   
"""
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(4000)
      # \* grap every thing in that directory that has the extinction jpg
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(4000)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(4000)


In [11]:
def preprocess(file_path):
    # Read image from file_path ( read the file as a byte object)"""
    # A bytes object is an immutable sequence of bytes, conceptually similar to a string. of a bytes object
    # is an unsigned int that satisfies 0 b 1111 _ 1111 >= x >= 0.
    # """
    byte_img = tf.io.read_file(file_path)
    # Load in the image
    img = tf.io.decode_jpeg(byte_img)
    # Preprocessing steps - resizing the image to be 100x100x3 channels (as it's a RGP image)
    img = tf.image.resize(img, (100, 100))
    # Scale the image to be between 0 and 1
    # Every single pixel value traditionally from 0 to 255 to be 0 to 1
    # that will help us in Gradient Descent
    img = img / 255.0
    # Return the image
    return img

In [12]:
# Allow us to iterate throw them all
"""
(anchor, positive) -> 1, 1, 1, 1, 1
(anchor, negative) -> 0, 0, 0, 0, 0
"""
# Create our labels
# hint tf.ones (len(anchor) will return a numpy array with the shape of 300(100x100x3) and dtype=float32
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
# Separate dataset for the negative
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
# the shape of data is ((), (), (), (tf.string , tf.string, tf.float32))
# Make the dataset of the positive and negative together in one big dataset
data = positives.concatenate(negatives)

In [13]:
# this function will take two images and preprocessing them
def preprocess_twin(input_image, validation_image, label):
    return preprocess(input_image), preprocess(validation_image), label


In [14]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)


In [15]:
"""
data shape will be : ((100, 100, None), (100, 100, None), ()), types( float32, float32, float32)
"""

# Training partition
# Here will take 70% of our data (500 * 70% = 350) as we can pass 350 directly
train_data = data.take(round(len(data) * .7))
# we will not input the images one by one we will input them by batches (16)
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# data shapes : ((None, 100, 100, None), (None, 100, 100, None), (None,)), types :(tf.float32, tf.float32, tf.float32)

# Testing partition
test_data = data.skip(round(len(data) * 0.7))
test_data = test_data.take(round(len(data) * 0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [16]:
# Making the Layers
def make_Embedding():
    inp = Input(shape=(100, 100, 3), name="input_image")

    # First block of the network
    conv1 = Conv2D(64, (10, 10), activation='relu')(inp)
    maxpooling1 = MaxPool2D(64, (2, 2), padding="same")(conv1)
    droup_out1 = Dropout(0.3)(maxpooling1)

    # Second block
    conv2 = Conv2D(128, (7, 7), activation="relu")(maxpooling1)
    maxpooling2 = MaxPool2D(64, (2, 2), padding="same")(conv2)
    droup_out2 = Dropout(0.3)(maxpooling2)

    # Third block
    conv3 = Conv2D(128, (4, 4), activation="relu")(maxpooling2)
    maxpooling3 = MaxPool2D(64, (2, 2), padding="same")(conv3)
    droup_out3 = Dropout(0.3)(maxpooling3)

    # Final embadding block
    conv4 = Conv2D(265, (4, 4), activation="relu")(droup_out3)
    flatten = Flatten()(conv4)
    dense1 = Dense(4096, activation="relu")(flatten)
    dense2 = Dense(1024, activation="sigmoid")(dense1)
    return Model(inputs=[inp], outputs=[dense2], name="Embedding")


In [17]:
embedding = make_Embedding()
embedding.summary()

Model: "Embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

In [18]:
embedding = make_Embedding()


In [19]:
# Siamese L1 Distance class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [20]:
def make_siamese_model():
    # Anchor image input in Neural Network
    input_image = Input(name='input_image', shape=(100, 100, 3))

    # Validation image in Neural Network
    validation_image = Input(name="validation", shape=(100, 100, 3))

    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classification
    classifier = Dense(1, activation='sigmoid')(distances)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")


In [21]:
siamese_network = make_siamese_model()
siamese_network.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation (InputLayer)        [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Embedding (Functional)         (None, 1024)         44501321    ['input_image[0][0]',            
                                                                  'validation[0][0]']

In [22]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)  

In [23]:
"""
Note : we can load the wights of this network for training another project by
model.load('path_to_checkpoint) this will load the pretrained weights into the existing model
"""
chekpoint_directory = './training_checkpoints'
# we will have our consistent format of ckpt
chekpoint_prefix = os.path.join(chekpoint_directory, 'ckpt')

checkpoint = tf.train.Checkpoint(opt=optimizer, siamese_network=siamese_network)

In [24]:
"""
@tf.function : Compiles a function into a callable TensorFlow graph
"""


# Build Train step function
@tf.function
def train(batch):
    # that's allow us to record and calculate the gradient descent (Backpropagation)
    with tf.GradientTape() as tape:
        # Get anchor and positive / negative image
        x = batch[:2]
        y = batch[2]
        # Forward pass
        # setting the training in our model to true will activate the layers to be trained ( like dropout)
        yhat = siamese_network(x, training=True)
        # Calculate Loss
        loss = binary_cross_loss(y, yhat)

        # Calculate gradients(Gradient Descent)
        grad = tape.gradient(loss, siamese_network.trainable_variables)
        # Calculate update weights and apply to siamese network
        # The optimizer here is calculating and propagating the new weights using Adam's optimisation algorithm
        optimizer.apply_gradients(zip(grad, siamese_network.trainable_variables))
        # Return Loss
        return loss

In [25]:
def train_loop(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS + 1):
        print(f"\n Epoch {epoch}/{EPOCHS}")
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object
        r = Recall()
        p = Precision()

        # Loop through each batch
        for index, batch in enumerate(data):
            
            
            # Run train function here
            loss = train(batch)
            yhat = siamese_network.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            progbar.update(index + 1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())   

        # Save the checkpoints every 10 epoch
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=chekpoint_prefix)


In [26]:
Epochs = 50

train_loop(train_data, Epochs)




 Epoch 1/50
113/350 [========>.....................] - ETA: 1:09:36

KeyboardInterrupt: 

In [2]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_network.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true, yhat)
print( r.result().numpy(), p.result().numpy())    

NameError: name 'Recall' is not defined

In [3]:
# Make prediction
y_hat = siamese_network.predict([test_input, test_val])

[1 if prediction > 0.5 else 0 for prediction in y_hat]

NameError: name 'siamese_network' is not defined

In [4]:
"""
# Note : we are doing that on a 1 batch in every iteration 
"""

# Creating a metric object
recall = Recall()

# Calculating the recall value
recall.update_state(y_true, y_hat)

# Return Recall Result
print(recall.result().numpy())

NameError: name 'Recall' is not defined

In [5]:
"""
for the Precision
"""
# Creating a metric object
precision = Precision()
# Calculating the recall value
precision.update_state(y_true, y_hat)

# Return Recall Result
print(precision.result().numpy())

NameError: name 'Precision' is not defined

In [6]:
"""
The subplot function allows us to render plots using a grid like format
The first parameter is the number of rows and the second one is the number of columns and 
the third one is the index    
"""
# Visualization the results
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
# As we know we can change the numbers that are passing in test_input or test_val
plt.imshow(test_input[0])
# As we know we can change the numbers that are passing in test_input or test_val
plt.subplot(1, 2, 2)
plt.imshow(test_val[0])
plt.show()

NameError: name 'plt' is not defined

In [35]:
# Save the wights of our siamese_network
siamese_network.save("Siamese_Model.h5")

In [36]:
"""
The load_weights() method can be used to load the weights also.The save_weights() method saves only
the weights, the save() method saves optimizers as well 
"""
# Reload model
# The second line we define our custom_objects so when we are building the kivy app we will need a reference on it
model = tf.keras.models.load_model("Siamese_Model.h5",
                                   custom_objects={'L1Dist': L1Dist,
                                                   "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy})


In [37]:
# Making prediction with our loaded model
model.predict([test_input, test_val])
model.summary() # we should see the summary and every layer in siamese_Network

1/1 [==============================] - 3s 3s/step
Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation (InputLayer)        [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Embedding (Functional)         (None, 4096)         40305993    ['input_image[0][0]',            
                                   

In [38]:
# Making verification function
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [40]:
# Varification
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv.imshow('Verification', frame)
    
    # Verification trigger
    if cv.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 123ms/step
False
1/1 [==============================] - 0s 108ms/step
False
1/1 [==============================] - 0s 110ms/step
True
1/1 [==============================] - 0s 107ms/step
True
1/1 [==============================] - 0s 110ms/step
False
1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 111ms/step
True


In [30]:
model